<a href="https://colab.research.google.com/github/karim-mammadov/Computer-Vision-My-projects/blob/main/Dog_Breeds_Image_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download darshanthakare/dog-breeds-image-dataset

In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile('/content/dog-breeds-image-dataset.zip','r') as zip_ref:
  zip_ref.extractall('/content')

In [ ]:
from PIL import Image
import os

allowed_extensions = ['.jpg', '.jpeg', '.png']

for root, dirs, files in os.walk(data_path):
    for file in files:
        file_lower = file.lower()
        if not any(file_lower.endswith(ext) for ext in allowed_extensions):
            img_path = os.path.join(root, file)
            try:
                # Attempt to open and save as jpg if possible, otherwise remove
                img = Image.open(img_path).convert('RGB')
                new_path = os.path.splitext(img_path)[0] + '.jpg'
                img.save(new_path, 'JPEG')
                os.remove(img_path)
            except Exception as e:
                # If conversion fails or it's not an image, just remove the problematic file
                print(f"Could not process or convert file {img_path}: {e}")
                os.remove(img_path)

In [ ]:
data_path = "/content/Dog Breeds Image Dataset"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input # Assuming you might use a pre-trained model later, this is a common preprocessor

# Define data generators with augmentation and validation split
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, # Optional: Use a preprocessing function if needed (e.g., for pre-trained models)
    validation_split=0.2,
    horizontal_flip=True, # Example augmentation
    rotation_range=20 # Example augmentation
)

# Create training and validation data generators
train_generator = datagen.flow_from_directory(
    data_path,
    target_size=imgs_size, # Use imgs_size as defined before
    batch_size=batch_size, # Use batch_size as defined before
    class_mode='categorical', # Use 'categorical' for one-hot encoding, or 'sparse' for integer labels
    subset='training',
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    data_path,
    target_size=imgs_size, # Use imgs_size as defined before
    batch_size=batch_size, # Use batch_size as defined before
    class_mode='categorical', # Use 'categorical' for one-hot encoding, or 'sparse' for integer labels
    subset='validation',
    shuffle=False
)

# Get the number of classes
num_classes = train_generator.num_classes
print(f"✅ Detected {num_classes} classes")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras import regularizers
import tensorflow as tf # Ensure tf is imported for tf.keras.layers.InputLayer


model = Sequential([
    tf.keras.layers.InputLayer(input_shape=(imgs_size[0], imgs_size[1], 3)), # Explicitly define input shape
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, (3,3),strides=1,activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3),strides=1,activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128, (3,3),strides=1,activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(256, (3,3),strides=1,activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dense(32, activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dense(num_classes, activation='softmax')

])

In [ ]:
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    epochs=5,
    validation_data=val_generator
)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input # Assuming you might use a pre-trained model later, this is a common preprocessor

# Define data generators with augmentation and validation split
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, # Optional: Use a preprocessing function if needed (e.g., for pre-trained models)
    validation_split=0.2,
    horizontal_flip=True, # Example augmentation
    rotation_range=20 # Example augmentation
)

# Create training and validation data generators
train_generator = datagen.flow_from_directory(
    data_path,
    target_size=imgs_size, # Use imgs_size as defined before
    batch_size=batch_size, # Use batch_size as defined before
    class_mode='categorical', # Use 'categorical' for one-hot encoding, or 'sparse' for integer labels
    subset='training',
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    data_path,
    target_size=imgs_size, # Use imgs_size as defined before
    batch_size=batch_size, # Use batch_size as defined before
    class_mode='categorical', # Use 'categorical' for one-hot encoding, or 'sparse' for integer labels
    subset='validation',
    shuffle=False
)

# Get the number of classes
num_classes = train_generator.num_classes
print(f"✅ Detected {num_classes} classes")